<a href="https://colab.research.google.com/github/PedroGuimaraesFerreira/Py_Projects/blob/master/covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1) Import necessary libraries



In [4]:
from bs4 import BeautifulSoup
import requests
import csv

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly.offline as py
import datetime
from datetime import date, timedelta
from sklearn.cluster import KMeans

import os

from geopy.geocoders import Nominatim
!pip install geocoder
import geocoder
import folium

from sklearn.cluster import KMeans

In [0]:
cases_df = pd.read_csv('covid_19_clean_complete.csv')